In [1]:
def pedir_numero(mensaje):
    while True:
        try:
            return int(input(mensaje))
        except ValueError:
            print("Esto no es un numero. ")

In [ ]:
def registro(diccionario):
    while True:
        usuario = input("Ingresa tu nombre de usuario: ")
        # Verificamos que el usuario no exista
        if usuario in diccionario:
            print("Este usuario ya existe en la base de datos.")
            continue
        else:
            password = input("Ingresa tu contrasena: ")
            diccionario[usuario] = password
            print("Usuario registrado exitosamente.\n")
            break

In [ ]:
def login(diccionario):
    while True:
        usuario = input("Ingresa tu nombre de usuario: ")
        # Verificamos que el usuario no exista        
        if usuario not in diccionario:
            print("Este usuario no existe en la base de datos.")
            continue
        else:
            for intento in range(3):
                password = input("Ingresa tu contraseña: ")
                if password == diccionario[usuario]:
                    print("¡login exitoso!")
                    return usuario
                else:
                    if intento < 2:
                        print(f"Contraseña invalida. Te quedan {2 - intento} intentos")
                    else:
                        print("Contraseña invalida.")
            print("Haz agotado tus intentos de contraseña")
            return None

In [4]:
def menu_acceso():
    print("\n=== Bienvenido ===")
    print("1. Registrarse")
    print("2. Iniciar sesión")
    print("3. Salir")
    while True:
        try:
            opcion_acceso = int(input("Que opcion quieres ejecutar: "))
            if opcion_acceso in [1, 2, 3]:
                print(f"\nHas seleccionado la opcion: {opcion_acceso}")
                break
            else:
                print("Por favor selecciona un numero entre 1 y 3.")
        except ValueError:
            print("Entrada no valida, por favor ingresa un numero")
    return (opcion_acceso)

In [ ]:
def menu():
    print("\nOperaciones: ")
    print("1. Suma \n2. Resta \n3. Multiplicacion \n4. Division \n5. Historial \n6. Salir")
    while True:
        try:
            opcion = int(input("Que opcion quieres ejecutar: "))
            if opcion in [1, 2, 3, 4, 5, 6]:
                print(f"\nHas seleccionado la opcion: {opcion}")
                break
            else:
                print("Por favor selecciona un numero entre 1 y 6.")
        except ValueError:
            print("Entrada no valida, por favor ingresa un numero")
    return (opcion)

In [ ]:
def operacion(opcion, a, b):
    operaciones = {
        1 : ("Suma", a + b),
        2 : ("Resta", a - b),
        3 : ("Multiplicacion", a * b),
        4 : ("Division", None if b == 0 else a / b)
    }
    nombre, resultado = operaciones[opcion]
    if resultado is None:
        print("No se puede dividir entre 0.")
    else:
        print(f"\nLa {nombre} de {a} y {b} es: {resultado}")

In [7]:
def repetir():
    while True:
        respuesta = input("\n¿Deseas volver al menu principal? (si/no): ").lower()
        if respuesta in ["si", "s"]:
            return True
        elif respuesta in ["no", "n"]:
            return False
        else:
            print("por favor ingresa 'si' o 'no'.")

In [8]:
def historial(lista, opcion, a, b):
    ''' lista: lista a modificar
        opcion: operacion a realizar
        a y b: numeros involucrados en la operacion '''
    
    operacion = {
        1 : ("+", (a + b)),
        2 : ("-", (a - b)),
        3 : ("*", (a * b)),
        4 : ("/", (a / b) if b != 0 else "Error")
    }        
    operador, resultado = operacion[opcion]
    
    lista.append(f" {a} {operador} {b} = {resultado}")

In [9]:
def mostrar_historial(lista):
    if len(lista) == 0:
        print("\nNo hay historial de operaciones por mostrar.")
    else:
        print("====Historial de operaciones====")
        for i, operacion in enumerate(lista, 1):
            print(f"{i}. {operacion}")

In [ ]:
def main():
    usuarios = {}
    historia = []
    usuario_actual = None

    while True:
        opcion_acceso = menu_acceso()
        if opcion_acceso == 1:
            registro(usuarios)
            usuario_actual = login(usuarios)
            if usuario_actual:
                break
        elif opcion_acceso == 2: 
            usuario_actual = login(usuarios)
            if usuario_actual:
                break
        else:
            print("¡Hasta Luego!")
            return

    print(f"\nBienvenido a la calculadora, {usuario_actual}")        
    while True:
        opcion = menu()
        if opcion == 5:
            mostrar_historial(historia)
        elif opcion == 6:
            print("¡Hasta luego!")
            break
        else:
            var_1 = pedir_numero("Ingresa un numero: ")
            var_2 = pedir_numero("Ingresa un numero: ")
            operacion(opcion, var_1, var_2)
            historial(historia, opcion, var_1, var_2)
            if not repetir():
                print("\nGracias por usar la calculadora. ¡Hasta luego!")
                break

In [11]:
main()




=== Bienvenido ===
1. Registrarse
2. Iniciar sesión
3. Salir

Has seleccionado la opcion: 1
Usuario registrado exitosamente.
Este usuario no existe en la base de datos.
Contraseña invalida. Te quedan 2 intentos
¡login exitoso!

Bienvenido a la calculadora, gamboyz

Operaciones: 
1. Suma 
2. Resta 
3. Multiplicacion 
4. Division 
5. Historial 
. Salir

Has seleccionado la opcion: 6
¡Hasta luego!
